In [4]:
# PASO 1: CONFIGURACIÓN DE NUESTRO LABORATORIO
# ===================================================================

import sqlite3
import pandas as pd

# Imagina que 'conectar' a una base de datos es como abrir la puerta de una bodega.
# Si el archivo 'empresa.db' no existe, Python lo crea por nosotros.
# Si ya existe, simplemente abrimos la puerta para trabajar con lo que hay adentro.
conn = sqlite3.connect('../Datos/empresa.db')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# El 'cursor' es nuestro operario o nuestro "brazo robótico" dentro de la bodega.
# No podemos mover las cajas (datos) directamente; le damos órdenes al cursor
# y él las ejecuta por nosotros.
cursor = conn.cursor()

In [ ]:
# --- Función Auxiliar Mágica ---
# Para no repetir código, creamos una función que toma una orden (nuestra consulta SQL),
# se la pasa al operario (el cursor), recoge los resultados y nos los muestra
# en una tabla bonita y fácil de leer (un DataFrame de pandas).
def ejecutar_sql(query, conexion=conn):
  """
  Toma una consulta SQL, la ejecuta y devuelve el resultado
  en una tabla de pandas para una visualización clara.
  """
  df = pd.read_sql_query(query, conexion)
  # Usamos display() porque en Colab presenta las tablas de forma más elegante.
  display(df)

In [ ]:
print("✅ ¡Laboratorio configurado! La puerta a 'empresa.db' está abierta y nuestro operario está listo.")

✅ ¡Laboratorio configurado! La puerta a 'empresa.db' está abierta y nuestro operario está listo.


In [ ]:
# ===================================================================
# PASO 2: EJECUTANDO COMANDOS DDL
# ===================================================================

# Usamos cursor.execute() porque estamos dando órdenes de construcción,
# no pidiendo que nos traigan datos para ver.

# 2.1. CREATE TABLE: Definiendo los planos.
# La Clave Primaria (PRIMARY KEY) es el DNI de cada fila: única e irrepetible.
# La Clave Foránea (FOREIGN KEY) es el pegamento que une nuestras tablas. Le dice
# a la columna 'id_depto' de Empleados que su valor DEBE existir en la columna
# 'id_depto' de la tabla Departamentos. ¡Así mantenemos la integridad!
print("🏗️  Levantando las estructuras de las tablas...")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Departamentos (
    id_depto INTEGER PRIMARY KEY,
    nombre_depto TEXT NOT NULL UNIQUE,
    region TEXT
);
""")

🏗️  Levantando las estructuras de las tablas...


In [ ]:
ejecutar_sql("Select * FROM Departamentos")

NameError: name 'ejecutar_sql' is not defined

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS Empleados (
    id_empleado INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    apellido TEXT,
    salario REAL CHECK(salario > 0),
    id_depto INTEGER,
    FOREIGN KEY (id_depto) REFERENCES Departamentos(id_depto)
);
""")

In [ ]:
# 2.2. ALTER TABLE: Una pequeña remodelación.
# Nos dimos cuenta de que es crucial saber cuándo se unió un empleado a la empresa.
try:
    print("🛠️  Remodelando la tabla 'Empleados' para agregar la columna 'fecha_ingreso'...")
    cursor.execute("ALTER TABLE Empleados ADD COLUMN fecha_ingreso DATE;")
except sqlite3.OperationalError as e:
    # Este bloque 'try/except' es para evitar errores si ejecutamos la celda más de una vez.
    print(f"⚠️  Advertencia: {e}. ¡Parece que la remodelación ya se había hecho!")

🛠️  Remodelando la tabla 'Empleados' para agregar la columna 'fecha_ingreso'...


In [ ]:
cursor.execute("ALTER TABLE Empleados ADD COLUMN fecha_ingreso DATE;")

OperationalError: duplicate column name: fecha_ingreso

In [ ]:
# ¡Fundamental! Como en un videojuego, después de hacer cambios importantes,
# hay que "guardar la partida". conn.commit() escribe permanentemente en el archivo
# de la base de datos todos los cambios estructurales que hemos ordenado.
conn.commit()
print("\n✅ ¡Planos finalizados y guardados! Las estructuras 'Departamentos' y 'Empleados' están listas.")


✅ ¡Planos finalizados y guardados! Las estructuras 'Departamentos' y 'Empleados' están listas.


In [ ]:
### Uso de DROP

cursor.execute("""
               CREATE TABLE toy (
    id_empleado INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    apellido TEXT,
    salario REAL CHECK(salario > 0),
    id_depto INTEGER,
    FOREIGN KEY (id_depto) REFERENCES Departamentos(id_depto)
);
               """)

In [ ]:
ejecutar_sql("SELECT * FROM toy")

,id_empleado,nombre,apellido,salario,id_depto


In [ ]:
cursor.execute(""" DROP TABLE IF EXISTS toy  """)

In [ ]:
ejecutar_sql("SELECT * FROM toy")

DatabaseError: Execution failed on sql 'SELECT * FROM toy': no such table: toy

In [ ]:
cursor.execute("INSERT INTO Departamentos (id_depto, nombre_depto, region) VALUES (0, 'Analitica','Norte')")

In [ ]:
ejecutar_sql("Select * FROM Departamentos")

,id_depto,nombre_depto,region
0,0,Analitica,Norte


In [ ]:
cursor.execute("INSERT INTO Departamentos (id_depto, nombre_depto, region) VALUES (10, 'Analitica_2','Norte')")

In [ ]:
ejecutar_sql("Select * FROM Departamentos")

,id_depto,nombre_depto,region
0,0,Analitica,Norte
1,10,Analitica_2,Norte


In [ ]:
cursor.execute("""INSERT INTO Departamentos (id_depto, nombre_depto, region) 
                    VALUES         
                        (1, 'Ventas', 'Norte'),
                        (2, 'Tecnología', 'Central'),
                        (3, 'Recursos Humanos', 'Central'),
                        (4, 'Marketing', 'Sur')""")

In [ ]:
ejecutar_sql("Select * FROM Departamentos")

,id_depto,nombre_depto,region
0,0,Analitica,Norte
1,1,Ventas,Norte
2,2,Tecnología,Central
3,3,Recursos Humanos,Central
4,4,Marketing,Sur
5,10,Analitica_2,Norte


In [ ]:
cursor.execute("""DELETE FROM Departamentos""")

In [ ]:
ejecutar_sql("Select * FROM Departamentos")

,id_depto,nombre_depto,region


In [ ]:
# ===================================================================
# PASO 3: EJECUTANDO COMANDOS DML
# ===================================================================

# 3.1. INSERT: Poblando nuestras tablas.
# Usar 'executemany' es como contratar un equipo de mudanza en lugar de mover
# una caja a la vez. Es mucho más eficiente para insertar múltiples filas.
try:
    print("🚚 Insertando datos en 'Departamentos'...")
    deptos = [
        (1, 'Ventas', 'Norte'),
        (2, 'Tecnología', 'Central'),
        (3, 'Recursos Humanos', 'Central'),
        (4, 'Marketing', 'Sur')
    ]
    cursor.executemany("INSERT INTO Departamentos (id_depto, nombre_depto, region) VALUES (?, ?, ?)", deptos)

    print("🚚 Insertando datos en 'Empleados'...")
    empleados = [
        (1, 'Ana', 'García', 50000, 2, '2022-01-15'),
        (2, 'Luis', 'Martínez', 75000, 2, '2021-11-20'),
        (3, 'Sofía', 'Hernández', 45000, 1, '2023-03-01'),
        (4, 'Carlos', 'Pérez', 48000, 1, '2022-08-10'),
        (5, 'Elena', 'Ruiz', 80000, 2, '2020-05-01'),
        (6, 'Mario', 'Jiménez', 35000, 3, '2023-05-15')
    ]
    cursor.executemany("INSERT INTO Empleados (id_empleado, nombre, apellido, salario, id_depto, fecha_ingreso) VALUES (?, ?, ?, ?, ?, ?)", empleados)
    conn.commit()
    print("\n✅ ¡Datos insertados y guardados con éxito!")
except sqlite3.IntegrityError:
    print("\n⚠️  Advertencia: Parece que estos datos ya fueron insertados en una ejecución anterior. ¡No se duplicarán gracias a nuestras reglas (constraints)!")

# 3.2. SELECT: Echemos un primer vistazo.
print("\n🔍 --- Inspeccionando la tabla Departamentos ---")
ejecutar_sql("SELECT * FROM Departamentos;")

print("\n🔍 --- Inspeccionando la tabla Empleados ---")
ejecutar_sql("SELECT * FROM Empleados;")

🚚 Insertando datos en 'Departamentos'...
🚚 Insertando datos en 'Empleados'...

✅ ¡Datos insertados y guardados con éxito!

🔍 --- Inspeccionando la tabla Departamentos ---


,id_depto,nombre_depto,region
0,1,Ventas,Norte
1,2,Tecnología,Central
2,3,Recursos Humanos,Central
3,4,Marketing,Sur



🔍 --- Inspeccionando la tabla Empleados ---


,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,1,Ana,García,50000.0,2,2022-01-15
1,2,Luis,Martínez,75000.0,2,2021-11-20
2,3,Sofía,Hernández,45000.0,1,2023-03-01
3,4,Carlos,Pérez,48000.0,1,2022-08-10
4,5,Elena,Ruiz,80000.0,2,2020-05-01
5,6,Mario,Jiménez,35000.0,3,2023-05-15


In [ ]:
# ===================================================================
# PASO 3 (CONTINUACIÓN): MODIFICANDO DATOS
# ===================================================================

# 3.3. UPDATE: Elena Ruiz ha sido promovida. ¡Aumentemos su salario!
# La cláusula WHERE es crucial: nos aseguramos de actualizar SOLO el registro de Elena.
print("✍️  Actualizando el salario de Elena Ruiz...")
cursor.execute("UPDATE Empleados SET salario = 82500 WHERE id_empleado = 5;")
conn.commit()

print("Verificando la actualización:")
ejecutar_sql("SELECT * FROM Empleados WHERE id_empleado = 5;")

✍️  Actualizando el salario de Elena Ruiz...
Verificando la actualización:


,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,5,Elena,Ruiz,82500.0,2,2020-05-01


In [ ]:
# 3.4. DELETE: Lamentablemente, Mario Jiménez ha dejado la empresa.
# De nuevo, la cláusula WHERE nos asegura que solo borremos a Mario.
print("\n🗑️  Eliminando al empleado Mario Jiménez...")
cursor.execute("DELETE FROM Empleados WHERE id_empleado = 6;")


🗑️  Eliminando al empleado Mario Jiménez...


In [ ]:
conn.commit()

print("Verificando la eliminación (la tabla no debería contener a Mario):")
ejecutar_sql("SELECT * FROM Empleados;")

Verificando la eliminación (la tabla no debería contener a Mario):


,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,1,Ana,García,50000.0,2,2022-01-15
1,2,Luis,Martínez,75000.0,2,2021-11-20
2,3,Sofía,Hernández,45000.0,1,2023-03-01
3,4,Carlos,Pérez,48000.0,1,2022-08-10
4,5,Elena,Ruiz,82500.0,2,2020-05-01


In [ ]:
print("\n🗑️  Eliminando al empleado Mario Jiménez...")
cursor.execute("DELETE FROM Empleados WHERE id_empleado = 5;")


🗑️  Eliminando al empleado Mario Jiménez...


In [ ]:
ejecutar_sql("SELECT * FROM Empleados;")

,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,1,Ana,García,50000.0,2,2022-01-15
1,2,Luis,Martínez,75000.0,2,2021-11-20
2,3,Sofía,Hernández,45000.0,1,2023-03-01
3,4,Carlos,Pérez,48000.0,1,2022-08-10


In [ ]:
conn.rollback()

In [ ]:
ejecutar_sql("SELECT * FROM Empleados;")

,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,1,Ana,García,50000.0,2,2022-01-15
1,2,Luis,Martínez,75000.0,2,2021-11-20
2,3,Sofía,Hernández,45000.0,1,2023-03-01
3,4,Carlos,Pérez,48000.0,1,2022-08-10
4,5,Elena,Ruiz,82500.0,2,2020-05-01


In [ ]:
nuevos_empleados=[   (6, 'Mario', 'Jiménez', 35000, 3, '2023-05-15'),
                     (7,'Julio','Jaramillo',80000,8,'2025-01-01')
    ]
cursor.executemany("INSERT INTO Empleados (id_empleado, nombre, apellido, salario, id_depto, fecha_ingreso) VALUES (?, ?, ?, ?, ?, ?)", nuevos_empleados)

ejecutar_sql('SELECT * FROM Empleados')

ejecutar_sql('SELECT * FROM Departamentos')

,id_empleado,nombre,apellido,salario,id_depto,fecha_ingreso
0,1,Ana,García,50000.0,2,2022-01-15
1,2,Luis,Martínez,75000.0,2,2021-11-20
2,3,Sofía,Hernández,45000.0,1,2023-03-01
3,4,Carlos,Pérez,48000.0,1,2022-08-10
4,5,Elena,Ruiz,82500.0,2,2020-05-01
5,6,Mario,Jiménez,35000.0,3,2023-05-15
6,7,Julio,Jaramillo,80000.0,8,2025-01-01


,id_depto,nombre_depto,region
0,1,Ventas,Norte
1,2,Tecnología,Central
2,3,Recursos Humanos,Central
3,4,Marketing,Sur


In [ ]:
# ===================================================================
# PASO 4: EJECUTANDO CONSULTAS AVANZADAS
# ===================================================================

# 4.1. JOINs: Cruzando información para obtener respuestas completas.
print("🌉 --- INNER JOIN: ¿Quién trabaja en qué departamento? ---")
query_inner = """
SELECT
    e.nombre,
    e.apellido,
    e.salario,
    e.fecha_ingreso,
    d.nombre_depto,
    d.region
FROM
    Empleados e
INNER JOIN
    Departamentos d ON e.id_depto = d.id_depto; -- La condición que forma el puente
"""
ejecutar_sql(query_inner)

🌉 --- INNER JOIN: ¿Quién trabaja en qué departamento? ---


,nombre,apellido,salario,fecha_ingreso,nombre_depto,region
0,Ana,García,50000.0,2022-01-15,Tecnología,Central
1,Luis,Martínez,75000.0,2021-11-20,Tecnología,Central
2,Sofía,Hernández,45000.0,2023-03-01,Ventas,Norte
3,Carlos,Pérez,48000.0,2022-08-10,Ventas,Norte
4,Elena,Ruiz,82500.0,2020-05-01,Tecnología,Central
5,Mario,Jiménez,35000.0,2023-05-15,Recursos Humanos,Central


In [ ]:
print("\n🌉 --- LEFT JOIN: ¿Qué departamentos no tienen empleados? ---")
# El LEFT JOIN nos trae TODO de la tabla de la izquierda (Departamentos) y solo lo que coincida de la derecha (Empleados).
# Veremos que 'Marketing' aparece, pero con datos de empleado vacíos (NULL), ¡porque nadie trabaja ahí todavía!
query_left = """
SELECT

    d.nombre_depto,
    d.region,
    e.nombre,
    e.apellido,
    e.salario,
    e.fecha_ingreso
FROM
    Departamentos d
LEFT JOIN
    Empleados e ON d.id_depto = e.id_depto;
"""
ejecutar_sql(query_left)


🌉 --- LEFT JOIN: ¿Qué departamentos no tienen empleados? ---


,nombre_depto,region,nombre,apellido,salario,fecha_ingreso
0,Ventas,Norte,Carlos,Pérez,48000.0,2022-08-10
1,Ventas,Norte,Sofía,Hernández,45000.0,2023-03-01
2,Tecnología,Central,Ana,García,50000.0,2022-01-15
3,Tecnología,Central,Elena,Ruiz,82500.0,2020-05-01
4,Tecnología,Central,Luis,Martínez,75000.0,2021-11-20
5,Recursos Humanos,Central,Mario,Jiménez,35000.0,2023-05-15
6,Marketing,Sur,None,None,NaN,None


In [ ]:
query_right = """
SELECT

    d.nombre_depto,
    d.region,
    e.nombre,
    e.apellido,
    e.salario,
    e.fecha_ingreso
FROM
    Departamentos d
RIGHT JOIN
    Empleados e ON d.id_depto = e.id_depto;
"""
ejecutar_sql(query_right)

,nombre_depto,region,nombre,apellido,salario,fecha_ingreso
0,Ventas,Norte,Sofía,Hernández,45000.0,2023-03-01
1,Ventas,Norte,Carlos,Pérez,48000.0,2022-08-10
2,Tecnología,Central,Ana,García,50000.0,2022-01-15
3,Tecnología,Central,Luis,Martínez,75000.0,2021-11-20
4,Tecnología,Central,Elena,Ruiz,82500.0,2020-05-01
5,Recursos Humanos,Central,Mario,Jiménez,35000.0,2023-05-15
6,None,None,Julio,Jaramillo,80000.0,2025-01-01


In [ ]:
query_outer = """
SELECT

    d.nombre_depto,
    d.region,
    e.nombre,
    e.apellido,
    e.salario,
    e.fecha_ingreso
FROM
    Departamentos d
FULL OUTER JOIN
    Empleados e ON d.id_depto = e.id_depto;
"""
ejecutar_sql(query_right)

,nombre_depto,region,nombre,apellido,salario,fecha_ingreso
0,Ventas,Norte,Sofía,Hernández,45000.0,2023-03-01
1,Ventas,Norte,Carlos,Pérez,48000.0,2022-08-10
2,Tecnología,Central,Ana,García,50000.0,2022-01-15
3,Tecnología,Central,Luis,Martínez,75000.0,2021-11-20
4,Tecnología,Central,Elena,Ruiz,82500.0,2020-05-01
5,Recursos Humanos,Central,Mario,Jiménez,35000.0,2023-05-15
6,None,None,Julio,Jaramillo,80000.0,2025-01-01


In [ ]:
query_sin_cte = """
SELECT
    e.nombre,
    e.apellido,
    e.salario
FROM
    Empleados e
WHERE
    e.salario > (SELECT AVG(salario) from Empleados)
ORDER BY
    e.salario DESC;
"""
ejecutar_sql(query_right)

,nombre_depto,region,nombre,apellido,salario,fecha_ingreso
0,Ventas,Norte,Sofía,Hernández,45000.0,2023-03-01
1,Ventas,Norte,Carlos,Pérez,48000.0,2022-08-10
2,Tecnología,Central,Ana,García,50000.0,2022-01-15
3,Tecnología,Central,Luis,Martínez,75000.0,2021-11-20
4,Tecnología,Central,Elena,Ruiz,82500.0,2020-05-01
5,Recursos Humanos,Central,Mario,Jiménez,35000.0,2023-05-15
6,None,None,Julio,Jaramillo,80000.0,2025-01-01


In [ ]:
ejecutar_sql("SELECT AVG(salario) as avg_sal FROM Empleados")

,avg_sal
0,59357.142857


In [ ]:
# 4.2. CTEs: Escribiendo código limpio y lógico.
# Pregunta: ¿Qué empleados ganan más que el promedio de toda la empresa?

print("🧑‍🍳 --- CTE: Encontrando empleados con salario superior al promedio ---")
# Primero, preparamos el ingrediente: calculamos el salario promedio en una CTE.
# Luego, cocinamos el plato principal: seleccionamos a los empleados comparándolos con nuestro ingrediente.
query_cte = """
WITH SalarioPromedio AS (
    SELECT AVG(salario) as avg_sal
    FROM Empleados
)

SELECT
    e.nombre,
    e.apellido,
    e.salario
FROM
    Empleados e, SalarioPromedio sp
WHERE
    e.salario > sp.avg_sal
ORDER BY
    e.salario DESC;
"""
ejecutar_sql(query_cte)

🧑‍🍳 --- CTE: Encontrando empleados con salario superior al promedio ---


,nombre,apellido,salario
0,Elena,Ruiz,82500.0
1,Julio,Jaramillo,80000.0
2,Luis,Martínez,75000.0


In [ ]:

# 4.3. Funciones de Ventana: Análisis en otro nivel.
# Pregunta: ¿Cuál es el ranking de salarios de los empleados DENTRO de su propio departamento?

print("🏆 --- Funciones de Ventana: Ranking de salarios por departamento ---")
# PARTITION BY d.nombre_depto: Crea una "ventana" o grupo separado para cada departamento.
# ORDER BY e.salario DESC: Dentro de cada ventana, ordena a los empleados de mayor a menor salario.
# RANK(): Asigna el ranking según ese orden. El cálculo se reinicia para cada nuevo departamento.
query_window = """
SELECT
    e.nombre,
    d.nombre_depto,
    e.salario,
    RANK() OVER (PARTITION BY d.nombre_depto ORDER BY e.salario DESC) as ranking_en_depto,
    SUM(e.salario) OVER (PARTITION BY d.nombre_depto ORDER BY e.salario DESC) as sum_en_depto
FROM
    Empleados e
JOIN
    Departamentos d ON e.id_depto = d.id_depto;
"""
ejecutar_sql(query_window)

🏆 --- Funciones de Ventana: Ranking de salarios por departamento ---


,nombre,nombre_depto,salario,ranking_en_depto,sum_en_depto
0,Mario,Recursos Humanos,35000.0,1,35000.0
1,Elena,Tecnología,82500.0,1,82500.0
2,Luis,Tecnología,75000.0,2,157500.0
3,Ana,Tecnología,50000.0,3,207500.0
4,Carlos,Ventas,48000.0,1,48000.0
5,Sofía,Ventas,45000.0,2,93000.0


In [ ]:
# Cerramos la conexión
conn.close()
print("Conexión a la base de datos cerrada.")

Conexión a la base de datos cerrada.


In [ ]:
conn = sqlite3.connect('Base_prueba')

In [ ]:
import pandas as pd
DF = pd.DataFrame([1,1,2,3,5,8,13,21,34])

In [ ]:
DF.to_sql('fibo', conn)

9

In [ ]:
cursor = conn.cursor()

In [ ]:
pd.read_sql('SELECT * FROM fibo', con=conn)

,index,0
0,0,1
1,1,1
2,2,2
3,3,3
4,4,5
5,5,8
6,6,13
7,7,21
8,8,34


In [ ]:
DF = pd.read_json("https://www.datos.gov.co/resource/d7zw-hpf4.json?$limit=100000")
DF

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,tipo_delito,cantidad
0,1996-01-01T00:00:00.000,5,ANTIOQUIA,5042,SANTAFE DE ANTIOQUIA,SECUESTRO SIMPLE,1
1,1996-01-01T00:00:00.000,25,CUNDINAMARCA,25290,FUSAGASUGA,SECUESTRO EXTORSIVO,1
2,1996-01-02T00:00:00.000,5,ANTIOQUIA,5038,ANGOSTURA,SECUESTRO EXTORSIVO,1
3,1996-01-02T00:00:00.000,68,SANTANDER,68001,BUCARAMANGA,SECUESTRO EXTORSIVO,1
4,1996-01-02T00:00:00.000,5,ANTIOQUIA,5088,BELLO,SECUESTRO EXTORSIVO,1
...,...,...,...,...,...,...,...
28015,2025-04-16T00:00:00.000,5,ANTIOQUIA,5440,MARINILLA,SECUESTRO EXTORSIVO,1
28016,2025-04-16T00:00:00.000,5,ANTIOQUIA,5440,MARINILLA,SECUESTRO EXTORSIVO,1
28017,2025-04-16T00:00:00.000,19,CAUCA,19001,POPAYAN,SECUESTRO EXTORSIVO,1
28018,2025-04-16T00:00:00.000,19,CAUCA,19001,POPAYAN,SECUESTRO EXTORSIVO,1


In [ ]:
import sqlite3

# Crear la conexión y guardar el DataFrame como tabla "secuestros"
conn = sqlite3.connect("secuestros.db")
DF.to_sql("secuestros", conn, if_exists="replace", index=False)
conn.close()

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("secuestros.db")

# Total de registros
pd.read_sql("SELECT COUNT(*) AS total_registros FROM secuestros", conn)

,total_registros
0,28020


In [ ]:
# Delitos por departamento
pd.read_sql("""
    SELECT departamento, COUNT(*) AS total
    FROM secuestros
    GROUP BY departamento
    ORDER BY total DESC
    LIMIT 10
""", conn)

,departamento,total
0,ANTIOQUIA,4697
1,CESAR,2123
2,VALLE DEL CAUCA,1965
3,BOGOTA D.C.,1553
4,SANTANDER,1466
5,META,1372
6,NORTE DE SANTANDER,1355
7,CUNDINAMARCA,1342
8,TOLIMA,1151
9,CAUCA,1088


In [ ]:
pd.read_sql("""
    SELECT strftime('%Y', fecha_hecho) AS anio, COUNT(*) AS total
    FROM secuestros
    WHERE fecha_hecho IS NOT NULL
    GROUP BY anio
    ORDER BY anio
""", conn)

,anio,total
0,1996,1038
1,1997,1624
2,1998,2860
3,1999,3205
4,2000,3572
5,2001,2917
6,2002,2882
7,2003,2123
8,2004,1440
9,2005,801


In [ ]:
conn.close()